In [10]:
import requests
import pandas as pd
import numpy as np
import time
import datetime
import sqlite3

In [11]:
def updateprice(datestr,dateupdate,con):

    #修改1
    datestr = datestr
    #修改2
    dateupdate = dateupdate

    urlDailyPrice = "http://www.twse.com.tw/exchangeReport/MI_INDEX?response=json&date={}&type=ALLBUT0999".format(datestr)
    html = requests.get(urlDailyPrice)

    html.encoding = "utf-8"
    dataInfo = html.json()

    colNames = ['交易日','證券代號','證券名稱','成交股數','成交筆數','成交金額','開盤價','最高價','最低價','收盤價','漲跌'
                ,'漲跌價差','最後揭示買價','最後揭示買量','最後揭示賣價','最後揭示賣量','本益比']

    if dataInfo['stat'] != "很抱歉，沒有符合條件的資料!" :

        orglist = dataInfo["data5"]
        listVals = orglist[:]

        count = 0
        for stockInfo in listVals:
            stockInfo[2] = int(stockInfo[2].replace(",",''))
            stockInfo[3] = stockInfo[3].replace(",",'')
            stockInfo[4] = stockInfo[4].replace(",",'')
            stockInfo[5] = stockInfo[5].replace(",",'')
            stockInfo[6] = stockInfo[6].replace(",",'')
            stockInfo[7] = stockInfo[7].replace(",",'')
            stockInfo[8] = stockInfo[8].replace(",",'')
            stockInfo[11] = stockInfo[11].replace(",",'')
            stockInfo[12] = stockInfo[12].replace(",",'')
            stockInfo[13] = stockInfo[13].replace(",",'')
            stockInfo[14] = stockInfo[14].replace(",",'')
            stockInfo[15] = stockInfo[15].replace(",",'')
            try:                
                stockInfo[5] = float(stockInfo[5])
            except ValueError:
                pass

            try:
                stockInfo[6] = float(stockInfo[6])
            except ValueError:
                pass

            try:
                stockInfo[7] = float(stockInfo[7])
            except ValueError:
                pass

            try:
                stockInfo[8] = float(stockInfo[8])
            except ValueError:
                pass

            upordown = stockInfo[9].find('color:green')
            if upordown != -1 :
                stockInfo[10] = float(stockInfo[10])*(-1.0)

            stockInfo.insert(0,dateupdate)


        df = pd.DataFrame.from_records(listVals,columns=colNames)
        df = df.drop('漲跌',axis=1)
        
        try:
            df.to_sql('price',con,if_exists='append',index =False)
            print('save ok')
        except Exception as e:
            print(e)
            pass       
    else:
        print('no data!!')

In [12]:
con = sqlite3.connect("stock.db")

searchdate = pd.date_range(start='2018-10-01',periods=5)
for d in searchdate:
    d1 = datetime.datetime.strftime(d,'%Y%m%d')
    d2 = datetime.datetime.strftime(d,'%Y-%m-%d')
    print('-----------------')
    print('search date : ',d2)
    updateprice(d1,d2,con)
    print('-----------------')
    
print('== ok ==')
con.commit()
con.close()
print('== close ==')

-----------------
search date :  2018-10-01
save ok
-----------------
-----------------
search date :  2018-10-02
save ok
-----------------
-----------------
search date :  2018-10-03
save ok
-----------------
-----------------
search date :  2018-10-04
save ok
-----------------
-----------------
search date :  2018-10-05
save ok
-----------------
== ok ==
== close ==
